<a href="https://colab.research.google.com/github/sreenithya2118/mini-project/blob/main/MiniProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpy

In [ ]:
class Host:
    def __init__(self, id, cpu_capacity, memory_capacity):
        self.id = id
        self.cpu_capacity = cpu_capacity
        self.memory_capacity = memory_capacity
        self.available_cpu = cpu_capacity
        self.available_memory = memory_capacity
        self.vms = []

    def allocate_resources(self, vm):
        if self.available_cpu >= vm.cpu and self.available_memory >= vm.memory:
            self.available_cpu -= vm.cpu
            self.available_memory -= vm.memory
            self.vms.append(vm)
            return True
        return False

    def release_resources(self, vm):
        self.available_cpu += vm.cpu
        self.available_memory += vm.memory
        self.vms.remove(vm)


In [ ]:
class VM:
    def __init__(self, id, cpu, memory):
        self.id = id
        self.cpu = cpu
        self.memory = memory
        self.host = None


In [ ]:
import random

def generate_vm_workload(env, arrival_rate, vm_list):
    while True:
        yield env.timeout(random.expovariate(arrival_rate))
        vm_id = len(vm_list) + 1
        cpu_demand = random.randint(1, 10)
        memory_demand = random.randint(1, 10)
        vm = VM(vm_id, cpu_demand, memory_demand)
        vm_list.append(vm)
        env.process(handle_vm_lifetime(env, vm))


In [ ]:
simulation_duration = 1000  # Total time units for the simulation
arrival_rate = 0.1  # Average arrival rate of VMs per time unit
min_cpu = 1
max_cpu = 10
min_memory = 1
max_memory = 10
#Q-learning parameters
alpha = 0.1  # Learning rate
gamma = 0.9  # Discount factor
epsilon = 0.1  # Exploration rate


In [ ]:
class QLearningAgent:
    def __init__(self, alpha, gamma, epsilon):
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.q_table = {}

    def get_state(self, host, vm):
        return (host.id, vm.id)

    def choose_action(self, state):
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(['migrate', 'stay'])
        else:
            return max(self.q_table.get(state, {'migrate': 0, 'stay': 0}), key=self.q_table.get(state, {'migrate': 0, 'stay': 0}).get)

    def update_q_value(self, state, action, reward, next_state):
        old_value = self.q_table.get(state, {}).get(action, 0)
        future_reward = max(self.q_table.get(next_state, {}).values(), default=0)
        new_value = old_value + self.alpha * (reward + self.gamma * future_reward - old_value)
        if state not in self.q_table:
            self.q_table[state] = {}
        self.q_table[state][action] = new_value


In [ ]:
def calculate_reward(host):
    utilization = (host.cpu_capacity - host.available_cpu) / host.cpu_capacity
    return utilization  # Simple reward based on CPU utilization


In [ ]:
def handle_vm_lifetime(env, vm):
    global hosts, agent
    # Find a host to allocate the VM
    for host in hosts:
        if host.allocate_resources(vm):
            vm.host = host
            break

    # VM is active for some time
    lifetime = random.randint(50, 150)
    yield env.timeout(lifetime)

    # Decide whether to migrate or not using Q-learning
    state = agent.get_state(vm.host, vm)
    action = agent.choose_action(state)

    if action == 'migrate':
        for host in hosts:
            if host != vm.host and host.allocate_resources(vm):
                vm.host.release_resources(vm)
                vm.host = host
                break

    reward = calculate_reward(vm.host)
    next_state = agent.get_state(vm.host, vm)
    agent.update_q_value(state, action, reward, next_state)

    # VM leaves the system
    vm.host.release_resources(vm)


In [ ]:
import simpy

env = simpy.Environment()
hosts = [Host(i, 100, 100) for i in range(10)]
vms = []
agent = QLearningAgent(alpha, gamma, epsilon)

env.process(generate_vm_workload(env, arrival_rate, vms))
env.run(until=simulation_duration)


In [ ]:
for host in hosts:
    print(f'Host {host.id} final utilization: {(host.cpu_capacity - host.available_cpu) / host.cpu_capacity:.2f}')


Host 0 final utilization: 0.81
Host 1 final utilization: 0.00
Host 2 final utilization: 0.00
Host 3 final utilization: 0.00
Host 4 final utilization: 0.00
Host 5 final utilization: 0.00
Host 6 final utilization: 0.00
Host 7 final utilization: 0.00
Host 8 final utilization: 0.00
Host 9 final utilization: 0.00


In [ ]:
import simpy
import random

class Host:
    def __init__(self, id, cpu_capacity, memory_capacity):
        self.id = id
        self.cpu_capacity = cpu_capacity
        self.memory_capacity = memory_capacity
        self.available_cpu = cpu_capacity
        self.available_memory = memory_capacity
        self.vms = []

    def allocate_resources(self, vm):
        if self.available_cpu >= vm.cpu and self.available_memory >= vm.memory:
            self.available_cpu -= vm.cpu
            self.available_memory -= vm.memory
            self.vms.append(vm)
            return True
        return False

    def release_resources(self, vm):
        self.available_cpu += vm.cpu
        self.available_memory += vm.memory
        self.vms.remove(vm)

class VM:
    def __init__(self, id, cpu, memory, start_time, end_time):
        self.id = id
        self.cpu = cpu
        self.memory = memory
        self.start_time = start_time
        self.end_time = end_time
        self.host = None

class QLearningAgent:
    def __init__(self, alpha, gamma, epsilon):
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.q_table = {}

    def get_state(self, host, vm):
        return (host.id, vm.id)

    def choose_action(self, state):
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(['migrate', 'stay'])
        else:
            return max(self.q_table.get(state, {'migrate': 0, 'stay': 0}), key=self.q_table.get(state, {'migrate': 0, 'stay': 0}).get)

    def update_q_value(self, state, action, reward, next_state):
        old_value = self.q_table.get(state, {}).get(action, 0)
        future_reward = max(self.q_table.get(next_state, {}).values(), default=0)
        new_value = old_value + self.alpha * (reward + self.gamma * future_reward - old_value)
        if state not in self.q_table:
            self.q_table[state] = {}
        self.q_table[state][action] = new_value

def calculate_reward(host):
    utilization = (host.cpu_capacity - host.available_cpu) / host.cpu_capacity
    return utilization  # Simple reward based on CPU utilization

def handle_vm_lifetime(env, vm, hosts, agent):
    for host in hosts:
        if host.allocate_resources(vm):
            vm.host = host
            break

    if vm.host is None:
        return  # No host could allocate the VM

    lifetime = vm.end_time - vm.start_time
    yield env.timeout(lifetime)

    state = agent.get_state(vm.host, vm)
    action = agent.choose_action(state)

    if action == 'migrate':
        for host in hosts:
            if host != vm.host and host.allocate_resources(vm):
                vm.host.release_resources(vm)
                vm.host = host
                break

    reward = calculate_reward(vm.host)
    next_state = agent.get_state(vm.host, vm)
    agent.update_q_value(state, action, reward, next_state)

    vm.host.release_resources(vm)

def generate_random_vm_workload(env, hosts, agent):
    while True:
        start_time = env.now
        end_time = start_time + random.randint(100, 1000)
        cpu_demand = random.uniform(10, 80)  # Increased range of CPU demand
        memory_demand = random.uniform(10, 80)  # Increased range of memory demand

        vm_id = start_time
        vm = VM(vm_id, cpu_demand, memory_demand, start_time, end_time)
        env.process(handle_vm_lifetime(env, vm, hosts, agent))
        yield env.timeout(random.randint(10, 100))

def print_host_utilization(hosts):
    for host in hosts:
        utilization = (host.cpu_capacity - host.available_cpu) / host.cpu_capacity
        print(f'Host {host.id} utilization: {utilization:.2f}')

# Simulation parameters
simulation_duration = 3600
alpha = 0.1
gamma = 0.9
epsilon = 0.1

env = simpy.Environment()
hosts = [Host(i, random.randint(80, 120), random.randint(80, 120)) for i in range(10)]  # Variance in host capacities
agent = QLearningAgent(alpha, gamma, epsilon)

env.process(generate_random_vm_workload(env, hosts, agent))
env.run(until=simulation_duration)

print_host_utilization(hosts)


Host 0 utilization: 0.54
Host 1 utilization: 0.46
Host 2 utilization: 0.51
Host 3 utilization: 0.60
Host 4 utilization: 0.76
Host 5 utilization: 0.00
Host 6 utilization: 0.12
Host 7 utilization: 0.47
Host 8 utilization: 0.68
Host 9 utilization: 0.00


In [ ]:
import random
import simpy

class Host:
    def __init__(self, id, cpu_capacity, memory_capacity):
        self.id = id
        self.cpu_capacity = cpu_capacity
        self.memory_capacity = memory_capacity
        self.available_cpu = cpu_capacity
        self.available_memory = memory_capacity
        self.vms = []

    def allocate_resources(self, vm):
        if self.available_cpu >= vm.cpu and self.available_memory >= vm.memory:
            self.available_cpu -= vm.cpu
            self.available_memory -= vm.memory
            self.vms.append(vm)
            print(f"VM {vm.id} allocated to Host {self.id}")
            return True
        return False

    def release_resources(self, vm):
        self.available_cpu += vm.cpu
        self.available_memory += vm.memory
        self.vms.remove(vm)
        print(f"VM {vm.id} released from Host {self.id}")

class VM:
    def __init__(self, id, cpu, memory, start_time, end_time):
        self.id = id
        self.cpu = cpu
        self.memory = memory
        self.start_time = start_time
        self.end_time = end_time
        self.host = None

class QLearningAgent:
    def __init__(self, alpha, gamma, epsilon):
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.q_table = {}

    def get_state(self, host, vm):
        return (host.id, vm.id)

    def choose_action(self, state):
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(['migrate', 'stay'])
        else:
            return max(self.q_table.get(state, {'migrate': 0, 'stay': 0}), key=self.q_table.get(state, {'migrate': 0, 'stay': 0}).get)

    def update_q_value(self, state, action, reward, next_state):
        old_value = self.q_table.get(state, {}).get(action, 0)
        future_reward = max(self.q_table.get(next_state, {}).values(), default=0)
        new_value = old_value + self.alpha * (reward + self.gamma * future_reward - old_value)
        if state not in self.q_table:
            self.q_table[state] = {}
        self.q_table[state][action] = new_value

def calculate_reward(host):
    utilization = (host.cpu_capacity - host.available_cpu) / host.cpu_capacity
    return utilization

def handle_vm_lifetime(env, vm, hosts, agent):
    for host in hosts:
        if host.allocate_resources(vm):
            vm.host = host
            break

    if vm.host is None:
        print(f"VM {vm.id} could not be allocated to any host")
        return

    print_host_utilization(hosts, "before migration")

    lifetime = vm.end_time - vm.start_time
    yield env.timeout(lifetime)

    state = agent.get_state(vm.host, vm)
    action = agent.choose_action(state)

    if action == 'migrate':
        for host in hosts:
            if host != vm.host and host.allocate_resources(vm):
                vm.host.release_resources(vm)
                vm.host = host
                break

    reward = calculate_reward(vm.host)
    next_state = agent.get_state(vm.host, vm)
    agent.update_q_value(state, action, reward, next_state)

    vm.host.release_resources(vm)

    print_host_utilization(hosts, "after migration")

def generate_vm_workload(env, hosts, agent):
    while True:
        start_time = env.now
        end_time = start_time + random.randint(5, 15)
        cpu_demand = random.uniform(10, 50)
        memory_demand = random.uniform(10, 50)

        vm_id = start_time
        vm = VM(vm_id, cpu_demand, memory_demand, start_time, end_time)
        env.process(handle_vm_lifetime(env, vm, hosts, agent))
        yield env.timeout(random.randint(1, 10))  # Wait for a random interval before creating the next VM

def print_host_utilization(hosts, migration_phase):
    print(f"Host utilization {migration_phase}:")
    for host in hosts:
        utilization = (host.cpu_capacity - host.available_cpu) / host.cpu_capacity
        print(f'Host {host.id} utilization: {utilization:.2f}')

# Simulation parameters
simulation_duration = 1000  # Short duration for quick test
alpha = 0.1
gamma = 0.9
epsilon = 0.1

# Initialize environment and components
env = simpy.Environment()
hosts = [Host(i, 100, 100) for i in range(5)]  # Reduced number of hosts for simplicity
agent = QLearningAgent(alpha, gamma, epsilon)

print("Starting simulation...")
env.process(generate_vm_workload(env, hosts, agent))
env.run(until=simulation_duration)
print("Simulation finished.")

print_host_utilization(hosts, "final")


Starting simulation...
VM 0 allocated to Host 0
Host utilization before migration:
Host 0 utilization: 0.45
Host 1 utilization: 0.00
Host 2 utilization: 0.00
Host 3 utilization: 0.00
Host 4 utilization: 0.00
VM 4 allocated to Host 0
Host utilization before migration:
Host 0 utilization: 0.57
Host 1 utilization: 0.00
Host 2 utilization: 0.00
Host 3 utilization: 0.00
Host 4 utilization: 0.00
VM 0 allocated to Host 1
VM 0 released from Host 0
VM 0 released from Host 1
Host utilization after migration:
Host 0 utilization: 0.12
Host 1 utilization: 0.00
Host 2 utilization: 0.00
Host 3 utilization: 0.00
Host 4 utilization: 0.00
VM 11 allocated to Host 0
Host utilization before migration:
Host 0 utilization: 0.35
Host 1 utilization: 0.00
Host 2 utilization: 0.00
Host 3 utilization: 0.00
Host 4 utilization: 0.00
VM 16 allocated to Host 0
Host utilization before migration:
Host 0 utilization: 0.55
Host 1 utilization: 0.00
Host 2 utilization: 0.00
Host 3 utilization: 0.00
Host 4 utilization: 0.00

In [ ]:
import random
import simpy

class Host:
    def __init__(self, id, cpu_capacity, memory_capacity):
        self.id = id
        self.cpu_capacity = cpu_capacity
        self.memory_capacity = memory_capacity
        self.available_cpu = cpu_capacity
        self.available_memory = memory_capacity
        self.vms = []

    def allocate_resources(self, vm):
        if self.available_cpu >= vm.cpu and self.available_memory >= vm.memory:
            self.available_cpu -= vm.cpu
            self.available_memory -= vm.memory
            self.vms.append(vm)
            print(f"VM {vm.id} allocated to Host {self.id}")
            return True
        return False

    def release_resources(self, vm):
        self.available_cpu += vm.cpu
        self.available_memory += vm.memory
        self.vms.remove(vm)
        print(f"VM {vm.id} released from Host {self.id}")

class VM:
    def __init__(self, id, cpu, memory, start_time, end_time):
        self.id = id
        self.cpu = cpu
        self.memory = memory
        self.start_time = start_time
        self.end_time = end_time
        self.host = None

class QLearningAgent:
    def __init__(self, alpha, gamma, epsilon):
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.q_table = {}

    def get_state(self, host, vm):
        return (host.id, vm.id)

    def choose_action(self, state):
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(['migrate', 'stay'])
        else:
            return max(self.q_table.get(state, {'migrate': 0, 'stay': 0}), key=self.q_table.get(state, {'migrate': 0, 'stay': 0}).get)

    def update_q_value(self, state, action, reward, next_state):
        old_value = self.q_table.get(state, {}).get(action, 0)
        future_reward = max(self.q_table.get(next_state, {}).values(), default=0)
        new_value = old_value + self.alpha * (reward + self.gamma * future_reward - old_value)
        if state not in self.q_table:
            self.q_table[state] = {}
        self.q_table[state][action] = new_value

def calculate_reward(host):
    utilization = (host.cpu_capacity - host.available_cpu) / host.cpu_capacity
    return utilization

def handle_vm_lifetime(env, vm, hosts, agent, migration_count, correct_migrations, total_actions):
    for host in hosts:
        if host.allocate_resources(vm):
            vm.host = host
            break

    if vm.host is None:
        print(f"VM {vm.id} could not be allocated to any host")
        return

    lifetime = vm.end_time - vm.start_time
    yield env.timeout(lifetime)

    state = agent.get_state(vm.host, vm)
    action = agent.choose_action(state)
    total_actions[0] += 1

    initial_reward = calculate_reward(vm.host)

    if action == 'migrate':
        for host in hosts:
            if host != vm.host and host.allocate_resources(vm):
                vm.host.release_resources(vm)
                vm.host = host
                migration_count[0] += 1
                break

    reward = calculate_reward(vm.host)
    next_state = agent.get_state(vm.host, vm)
    agent.update_q_value(state, action, reward, next_state)

    if reward > initial_reward:
        correct_migrations[0] += 1

    vm.host.release_resources(vm)

def generate_vm_workload(env, hosts, agent, migration_count, correct_migrations, total_actions):
    while True:
        start_time = env.now
        end_time = start_time + random.randint(5, 15)
        cpu_demand = random.uniform(10, 50)
        memory_demand = random.uniform(10, 50)

        vm_id = start_time
        vm = VM(vm_id, cpu_demand, memory_demand, start_time, end_time)
        env.process(handle_vm_lifetime(env, vm, hosts, agent, migration_count, correct_migrations, total_actions))
        yield env.timeout(random.randint(1, 10))  # Wait for a random interval before creating the next VM

def calculate_efficiency(hosts, total_time, migration_count, correct_migrations, total_actions):
    total_utilization = sum((host.cpu_capacity - host.available_cpu) / host.cpu_capacity for host in hosts) / len(hosts)
    print(f"Average Host Utilization: {total_utilization:.2f}")
    print(f"Number of Migrations: {migration_count[0]}")
    efficiency = total_utilization - (migration_count[0] / total_time)  # Simplified efficiency calculation
    accuracy = correct_migrations[0] / total_actions[0] if total_actions[0] > 0 else 0
    print(f"Efficiency: {efficiency:.2f}")
    print(f"Accuracy: {accuracy:.2f}")

def print_host_utilization(hosts):
    for host in hosts:
        utilization = (host.cpu_capacity - host.available_cpu) / host.cpu_capacity
        print(f'Host {host.id} utilization: {utilization:.2f}')

# Simulation parameters
simulation_duration = 1000  # Short duration for quick test
alpha = 0.1
gamma = 0.9
epsilon = 0.1

# Initialize environment and components
env = simpy.Environment()
hosts = [Host(i, 100, 100) for i in range(5)]  # Reduced number of hosts for simplicity
agent = QLearningAgent(alpha, gamma, epsilon)
migration_count = [0]  # Using a list to pass the migration count by reference
correct_migrations = [0]  # Track correct migrations
total_actions = [0]  # Track total actions

print("Starting simulation...")
env.process(generate_vm_workload(env, hosts, agent, migration_count, correct_migrations, total_actions))
env.run(until=simulation_duration)
print("Simulation finished.")

print_host_utilization(hosts)
calculate_efficiency(hosts, simulation_duration, migration_count, correct_migrations, total_actions)


Starting simulation...
VM 0 allocated to Host 0
VM 6 allocated to Host 0
VM 0 allocated to Host 1
VM 0 released from Host 0
VM 0 released from Host 1
VM 11 allocated to Host 0
VM 16 allocated to Host 0
VM 11 allocated to Host 1
VM 11 released from Host 0
VM 11 released from Host 1
VM 18 allocated to Host 1
VM 6 allocated to Host 1
VM 6 released from Host 0
VM 6 released from Host 1
VM 19 allocated to Host 0
VM 20 allocated to Host 1
VM 24 allocated to Host 1
VM 19 allocated to Host 2
VM 19 released from Host 0
VM 19 released from Host 2
VM 16 allocated to Host 2
VM 16 released from Host 0
VM 16 released from Host 2
VM 18 allocated to Host 0
VM 18 released from Host 1
VM 18 released from Host 0
VM 20 allocated to Host 0
VM 20 released from Host 1
VM 20 released from Host 0
VM 33 allocated to Host 0
VM 24 allocated to Host 0
VM 24 released from Host 1
VM 24 released from Host 0
VM 41 allocated to Host 0
VM 33 allocated to Host 1
VM 33 released from Host 0
VM 33 released from Host 1
VM 44

In [ ]:
pip install google-cloud-monitoring


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.9/344.9 kB 6.4 MB/s eta 0:00:00


In [ ]:
!pip install simpy

In [ ]:
!pip install google-cloud-monitoring simpy


In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/qwiklabs-gcp-03-103a595de79c-f295ae08711b.json"


In [ ]:
import datetime
from google.cloud import monitoring_v3
import simpy
import random

# Google Cloud Monitoring client
client = monitoring_v3.MetricServiceClient()

# Function to fetch real-time metrics from Google Cloud Monitoring
def fetch_cloud_metrics(project_id, instance_id):
    now = datetime.datetime.now(datetime.timezone.utc)
    interval = monitoring_v3.TimeInterval(
        {
            "end_time": now,
            "start_time": now - datetime.timedelta(minutes=5),
        }
    )

    results = client.list_time_series(
        request={
            "name": f"projects/{project_id}",
            "filter": f'metric.type="compute.googleapis.com/instance/cpu/utilization" AND resource.labels.instance_id="{instance_id}"',
            "interval": interval,
            "view": monitoring_v3.ListTimeSeriesRequest.TimeSeriesView.FULL,
        }
    )

    datapoints = []
    for result in results:
        for point in result.points:
            datapoints.append({
                "timestamp": point.interval.end_time,
                "average": point.value.double_value
            })

    return datapoints

class Host:
    def __init__(self, id, cpu_capacity, memory_capacity):
        self.id = id
        self.cpu_capacity = cpu_capacity
        self.memory_capacity = memory_capacity
        self.available_cpu = cpu_capacity
        self.available_memory = memory_capacity
        self.vms = []

    def allocate_resources(self, vm):
        if self.available_cpu >= vm.cpu and self.available_memory >= vm.memory:
            self.available_cpu -= vm.cpu
            self.available_memory -= vm.memory
            self.vms.append(vm)
            return True
        return False

    def release_resources(self, vm):
        self.available_cpu += vm.cpu
        self.available_memory += vm.memory
        self.vms.remove(vm)

class VM:
    def __init__(self, id, cpu, memory, start_time, end_time):
        self.id = id
        self.cpu = cpu
        self.memory = memory
        self.start_time = start_time
        self.end_time = end_time
        self.host = None

class QLearningAgent:
    def __init__(self, alpha, gamma, epsilon):
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.q_table = {}

    def get_state(self, host, vm):
        return (host.id, vm.id)

    def choose_action(self, state):
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(['migrate', 'stay'])
        else:
            return max(self.q_table.get(state, {'migrate': 0, 'stay': 0}), key=self.q_table.get(state, {'migrate': 0, 'stay': 0}).get)

    def update_q_value(self, state, action, reward, next_state):
        old_value = self.q_table.get(state, {}).get(action, 0)
        future_reward = max(self.q_table.get(next_state, {}).values(), default=0)
        new_value = old_value + self.alpha * (reward + self.gamma * future_reward - old_value)
        if state not in self.q_table:
            self.q_table[state] = {}
        self.q_table[state][action] = new_value


In [ ]:
import datetime
from google.cloud import monitoring_v3
import simpy
import random

# Google Cloud Monitoring client
client = monitoring_v3.MetricServiceClient()

# Function to fetch real-time metrics from Google Cloud Monitoring
def fetch_cloud_metrics(project_id, instance_id):
    now = datetime.datetime.now(datetime.timezone.utc)
    interval = monitoring_v3.TimeInterval(
        {
            "end_time": now,
            "start_time": now - datetime.timedelta(minutes=5),
        }
    )

    results = client.list_time_series(
        request={
            "name": f"projects/{project_id}",
            "filter": f'metric.type="compute.googleapis.com/instance/cpu/utilization" AND resource.labels.instance_id="{instance_id}"',
            "interval": interval,
            "view": monitoring_v3.ListTimeSeriesRequest.TimeSeriesView.FULL,
        }
    )

    datapoints = []
    for result in results:
        for point in result.points:
            datapoints.append({
                "timestamp": point.interval.end_time,
                "average": point.value.double_value
            })

    return datapoints

class Host:
    def __init__(self, id, cpu_capacity, memory_capacity):
        self.id = id
        self.cpu_capacity = cpu_capacity
        self.memory_capacity = memory_capacity
        self.available_cpu = cpu_capacity
        self.available_memory = memory_capacity
        self.vms = []

    def allocate_resources(self, vm):
        if self.available_cpu >= vm.cpu and self.available_memory >= vm.memory:
            self.available_cpu -= vm.cpu
            self.available_memory -= vm.memory
            self.vms.append(vm)
            return True
        return False

    def release_resources(self, vm):
        self.available_cpu += vm.cpu
        self.available_memory += vm.memory
        self.vms.remove(vm)

class VM:
    def __init__(self, id, cpu, memory, start_time, end_time):
        self.id = id
        self.cpu = cpu
        self.memory = memory
        self.start_time = start_time
        self.end_time = end_time
        self.host = None

class QLearningAgent:
    def __init__(self, alpha, gamma, epsilon):
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.q_table = {}

    def get_state(self, host, vm):
        return (host.id, vm.id)

    def choose_action(self, state):
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(['migrate', 'stay'])
        else:
            return max(self.q_table.get(state, {'migrate': 0, 'stay': 0}), key=self.q_table.get(state, {'migrate': 0, 'stay': 0}).get)

    def update_q_value(self, state, action, reward, next_state):
        old_value = self.q_table.get(state, {}).get(action, 0)
        future_reward = max(self.q_table.get(next_state, {}).values(), default=0)
        new_value = old_value + self.alpha * (reward + self.gamma * future_reward - old_value)
        if state not in self.q_table:
            self.q_table[state] = {}
        self.q_table[state][action] = new_value


In [ ]:
def calculate_reward(host):
    utilization = (host.cpu_capacity - host.available_cpu) / host.cpu_capacity
    return utilization

def handle_vm_lifetime(env, vm, hosts, agent):
    for host in hosts:
        if host.allocate_resources(vm):
            vm.host = host
            break

    if vm.host is None:
        return

    lifetime = vm.end_time - vm.start_time
    yield env.timeout(lifetime)

    state = agent.get_state(vm.host, vm)
    action = agent.choose_action(state)

    if action == 'migrate':
        for host in hosts:
            if host != vm.host and host.allocate_resources(vm):
                vm.host.release_resources(vm)
                vm.host = host
                break

    reward = calculate_reward(vm.host)
    next_state = agent.get_state(vm.host, vm)
    agent.update_q_value(state, action, reward, next_state)

    vm.host.release_resources(vm)


In [ ]:
def generate_vm_workload(env, project_id, instance_id, hosts, agent):
    while True:
        metrics = fetch_cloud_metrics(project_id, instance_id)
        for metric in metrics:
            start_time = metric['timestamp'].timestamp()
            end_time = start_time + 3  # Assuming 5-minute intervals
            cpu_demand = metric['average']
            memory_demand = random.uniform(0.1, 1.0) * 100

            vm_id = int(start_time)
            vm = VM(vm_id, cpu_demand, memory_demand, start_time, end_time)
            env.process(handle_vm_lifetime(env, vm, hosts, agent))
            yield env.timeout(6)  # Wait for the next interval to fetch new metrics


In [ ]:
def print_host_utilization(hosts):
    for host in hosts:
        utilization = (host.cpu_capacity - host.available_cpu) / host.cpu_capacity
        print(f'Host {host.id} utilization: {utilization:.2f}')


In [ ]:
# Simulation parameters
simulation_duration = 10  # Short duration for quick test
alpha = 0.1
gamma = 0.9
epsilon = 0.1

# Initialize environment and components
env = simpy.Environment()
hosts = [Host(i, 100, 100) for i in range(3)]  # Reduced number of hosts for simplicity
agent = QLearningAgent(alpha, gamma, epsilon)

# Example project ID and instance ID
project_id = 'qwiklabs-gcp-03-103a595de79c'
instance_id = 'instance-20240607-152435'

env.process(generate_vm_workload(env, project_id, instance_id, hosts, agent))
env.run(until=simulation_duration)

print_host_utilization(hosts)


PermissionDenied: 403 Project #1080153336926 has been deleted.

In [ ]:
import datetime
from google.cloud import monitoring_v3
import simpy
import random
import os

# Set up Google Cloud credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/qwiklabs-gcp-03-103a595de79c-f295ae08711b.json"

# Google Cloud Monitoring client
client = monitoring_v3.MetricServiceClient()

# Function to fetch real-time metrics from Google Cloud Monitoring
def fetch_cloud_metrics(project_id, instance_id):
    now = datetime.datetime.now(datetime.timezone.utc)
    interval = monitoring_v3.TimeInterval(
        {
            "end_time": now,
            "start_time": now - datetime.timedelta(minutes=5),
        }
    )

    results = client.list_time_series(
        request={
            "name": f"projects/{project_id}",
            "filter": f'metric.type="compute.googleapis.com/instance/cpu/utilization" AND resource.labels.instance_id="{instance_id}"',
            "interval": interval,
            "view": monitoring_v3.ListTimeSeriesRequest.TimeSeriesView.FULL,
        }
    )

    datapoints = []
    for result in results:
        for point in result.points:
            datapoints.append({
                "timestamp": point.interval.end_time,
                "average": point.value.double_value
            })

    return datapoints

class Host:
    def __init__(self, id, cpu_capacity, memory_capacity):
        self.id = id
        self.cpu_capacity = cpu_capacity
        self.memory_capacity = memory_capacity
        self.available_cpu = cpu_capacity
        self.available_memory = memory_capacity
        self.vms = []

    def allocate_resources(self, vm):
        if self.available_cpu >= vm.cpu and self.available_memory >= vm.memory:
            self.available_cpu -= vm.cpu
            self.available_memory -= vm.memory
            self.vms.append(vm)
            print(f"VM {vm.id} allocated to Host {self.id}")
            return True
        return False

    def release_resources(self, vm):
        self.available_cpu += vm.cpu
        self.available_memory += vm.memory
        self.vms.remove(vm)
        print(f"VM {vm.id} released from Host {self.id}")

class VM:
    def __init__(self, id, cpu, memory, start_time, end_time):
        self.id = id
        self.cpu = cpu
        self.memory = memory
        self.start_time = start_time
        self.end_time = end_time
        self.host = None

class QLearningAgent:
    def __init__(self, alpha, gamma, epsilon):
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.q_table = {}

    def get_state(self, host, vm):
        return (host.id, vm.id)

    def choose_action(self, state):
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(['migrate', 'stay'])
        else:
            return max(self.q_table.get(state, {'migrate': 0, 'stay': 0}), key=self.q_table.get(state, {'migrate': 0, 'stay': 0}).get)

    def update_q_value(self, state, action, reward, next_state):
        old_value = self.q_table.get(state, {}).get(action, 0)
        future_reward = max(self.q_table.get(next_state, {}).values(), default=0)
        new_value = old_value + self.alpha * (reward + self.gamma * future_reward - old_value)
        if state not in self.q_table:
            self.q_table[state] = {}
        self.q_table[state][action] = new_value

def calculate_reward(host):
    utilization = (host.cpu_capacity - host.available_cpu) / host.cpu_capacity
    return utilization

def handle_vm_lifetime(env, vm, hosts, agent):
    for host in hosts:
        if host.allocate_resources(vm):
            vm.host = host
            break

    if vm.host is None:
        print(f"VM {vm.id} could not be allocated to any host")
        return

    lifetime = vm.end_time - vm.start_time
    yield env.timeout(lifetime)

    state = agent.get_state(vm.host, vm)
    action = agent.choose_action(state)

    if action == 'migrate':
        for host in hosts:
            if host != vm.host and host.allocate_resources(vm):
                vm.host.release_resources(vm)
                vm.host = host
                break

    reward = calculate_reward(vm.host)
    next_state = agent.get_state(vm.host, vm)
    agent.update_q_value(state, action, reward, next_state)

    vm.host.release_resources(vm)

def generate_vm_workload(env, project_id, instance_id, hosts, agent):
    while True:
        print("Fetching metrics...")
        metrics = fetch_cloud_metrics(project_id, instance_id)
        if not metrics:
            print("No metrics fetched, retrying in 5 seconds...")
            yield env.timeout(5)  # Retry after a short delay
            continue

        for metric in metrics:
            start_time = metric['timestamp'].timestamp()
            end_time = start_time + 3  # Assuming 5-minute intervals
            cpu_demand = metric['average']
            memory_demand = 50  # Fixed memory demand for simplicity

            vm_id = int(start_time)
            vm = VM(vm_id, cpu_demand, memory_demand, start_time, end_time)
            env.process(handle_vm_lifetime(env, vm, hosts, agent))
            yield env.timeout(6)  # Wait for the next interval to fetch new metrics

def print_host_utilization(hosts):
    for host in hosts:
        utilization = (host.cpu_capacity - host.available_cpu) / host.cpu_capacity
        print(f'Host {host.id} utilization: {utilization:.2f}')

# Simulation parameters
simulation_duration = 10  # Short duration for quick test
alpha = 0.1
gamma = 0.9
epsilon = 0.1

# Initialize environment and components
env = simpy.Environment()
hosts = [Host(i, 100, 100) for i in range(3)]  # Reduced number of hosts for simplicity
agent = QLearningAgent(alpha, gamma, epsilon)

# Example project ID and instance ID
project_id = 'qwiklabs-gcp-03-103a595de79c'
instance_id = 'instance-20240607-152435'

print("Starting simulation...")
env.process(generate_vm_workload(env, project_id, instance_id, hosts, agent))
env.run(until=simulation_duration)
print("Simulation finished.")

print_host_utilization(hosts)


Starting simulation...
Fetching metrics...


PermissionDenied: 403 Project #1080153336926 has been deleted.

In [ ]:
import datetime
import random
import os
import simpy
from azure.identity import DefaultAzureCredential
from azure.monitor.query import MetricsQueryClient, MetricAggregationType

# Set up Azure credentials
credential = DefaultAzureCredential()
client = MetricsQueryClient(credential)

# Function to fetch real-time metrics from Azure Monitor
def fetch_cloud_metrics(subscription_id, resource_group, vm_name):
    try:
        # Define the resource ID
        resource_id = (
            f"/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.Compute/virtualMachines/{vm_name}"
        )

        # Query metrics
        response = client.query(
            resource_id,
            metric_names=["Percentage CPU"],
            timespan=datetime.timedelta(minutes=5),
            interval="PT1M",
            aggregations=[MetricAggregationType.AVERAGE]
        )

        datapoints = []
        for result in response.metrics[0].timeseries[0].data:
            if result.average is not None:
                datapoints.append({
                    "timestamp": result.timestamp,
                    "average": result.average
                })

        return datapoints
    except Exception as e:
        print(f"Error fetching metrics: {e}")
        return []

class Host:
    def __init__(self, id, cpu_capacity, memory_capacity):
        self.id = id
        self.cpu_capacity = cpu_capacity
        self.memory_capacity = memory_capacity
        self.available_cpu = cpu_capacity
        self.available_memory = memory_capacity
        self.vms = []

    def allocate_resources(self, vm):
        if self.available_cpu >= vm.cpu and self.available_memory >= vm.memory:
            self.available_cpu -= vm.cpu
            self.available_memory -= vm.memory
            self.vms.append(vm)
            print(f"VM {vm.id} allocated to Host {self.id}")
            return True
        return False

    def release_resources(self, vm):
        self.available_cpu += vm.cpu
        self.available_memory += vm.memory
        self.vms.remove(vm)
        print(f"VM {vm.id} released from Host {self.id}")

class VM:
    def __init__(self, id, cpu, memory, start_time, end_time):
        self.id = id
        self.cpu = cpu
        self.memory = memory
        self.start_time = start_time
        self.end_time = end_time
        self.host = None

class QLearningAgent:
    def __init__(self, alpha, gamma, epsilon):
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.q_table = {}

    def get_state(self, host, vm):
        return (host.id, vm.id)

    def choose_action(self, state):
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(['migrate', 'stay'])
        else:
            return max(self.q_table.get(state, {'migrate': 0, 'stay': 0}), key=self.q_table.get(state, {'migrate': 0, 'stay': 0}).get)

    def update_q_value(self, state, action, reward, next_state):
        old_value = self.q_table.get(state, {}).get(action, 0)
        future_reward = max(self.q_table.get(next_state, {}).values(), default=0)
        new_value = old_value + self.alpha * (reward + self.gamma * future_reward - old_value)
        if state not in self.q_table:
            self.q_table[state] = {}
        self.q_table[state][action] = new_value

def calculate_reward(host):
    utilization = (host.cpu_capacity - host.available_cpu) / host.cpu_capacity
    return utilization

def handle_vm_lifetime(env, vm, hosts, agent):
    for host in hosts:
        if host.allocate_resources(vm):
            vm.host = host
            break

    if vm.host is None:
        print(f"VM {vm.id} could not be allocated to any host")
        return

    lifetime = vm.end_time - vm.start_time
    yield env.timeout(lifetime)

    state = agent.get_state(vm.host, vm)
    action = agent.choose_action(state)

    if action == 'migrate':
        for host in hosts:
            if host != vm.host and host.allocate_resources(vm):
                vm.host.release_resources(vm)
                vm.host = host
                break

    reward = calculate_reward(vm.host)
    next_state = agent.get_state(vm.host, vm)
    agent.update_q_value(state, action, reward, next_state)

    vm.host.release_resources(vm)

def generate_vm_workload(env, subscription_id, resource_group, vm_name, hosts, agent):
    while True:
        print("Fetching metrics...")
        metrics = fetch_cloud_metrics(subscription_id, resource_group, vm_name)
        if not metrics:
            print("No metrics fetched, using random data...")
            metrics = [{
                "timestamp": datetime.datetime.now(datetime.timezone.utc),
                "average": random.uniform(0.1, 0.9)
            }]

        for metric in metrics:
            start_time = metric['timestamp'].timestamp()
            end_time = start_time + 15  # Reduced interval for more real-time simulation
            cpu_demand = metric['average']
            memory_demand = 50  # Fixed memory demand for simplicity

            vm_id = int(start_time)
            vm = VM(vm_id, cpu_demand, memory_demand, start_time, end_time)
            env.process(handle_vm_lifetime(env, vm, hosts, agent))
            yield env.timeout(10)  # Wait for the next interval to fetch new metrics

def print_host_utilization(hosts):
    for host in hosts:
        utilization = (host.cpu_capacity - host.available_cpu) / host.cpu_capacity
        print(f'Host {host.id} utilization: {utilization:.2f}')

# Simulation parameters
simulation_duration = 1000  # Short duration for quick test
alpha = 0.1
gamma = 0.9
epsilon = 0.1

# Initialize environment and components
env = simpy.Environment()
hosts = [Host(i, 100, 100) for i in range(5)]  # Reduced number of hosts for simplicity
agent = QLearningAgent(alpha, gamma, epsilon)

# Example subscription ID, resource group, and VM name
subscription_id = 'd89493f7-fd93-418b-a882-dd9d2cb7c099'
resource_group = 'cat2_group'
vm_name = 'Hhi2003'

print("Starting simulation...")
env.process(generate_vm_workload(env, subscription_id, resource_group, vm_name, hosts, agent))
env.run(until=simulation_duration)
print("Simulation finished.")

print_host_utilization(hosts)


Starting simulation...
Fetching metrics...
Error fetching metrics: 'MetricsQueryClient' object has no attribute 'query'
No metrics fetched, using random data...
VM 1717781748 allocated to Host 0
Fetching metrics...
Error fetching metrics: 'MetricsQueryClient' object has no attribute 'query'
No metrics fetched, using random data...
VM 1717781748 allocated to Host 0
VM 1717781748 allocated to Host 1
VM 1717781748 released from Host 0
VM 1717781748 released from Host 1
Fetching metrics...
Error fetching metrics: 'MetricsQueryClient' object has no attribute 'query'
No metrics fetched, using random data...
VM 1717781748 allocated to Host 0
VM 1717781748 allocated to Host 1
VM 1717781748 released from Host 0
VM 1717781748 released from Host 1
Fetching metrics...
Error fetching metrics: 'MetricsQueryClient' object has no attribute 'query'
No metrics fetched, using random data...
VM 1717781748 allocated to Host 0
VM 1717781748 allocated to Host 1
VM 1717781748 released from Host 0
VM 171778174

In [ ]:
import datetime
import random
import os
import simpy
from azure.identity import DefaultAzureCredential
from azure.monitor.query import MetricsQueryClient, MetricAggregationType

# Set up Azure credentials
credential = DefaultAzureCredential()
client = MetricsQueryClient(credential)

# Function to fetch real-time metrics from Azure Monitor
def fetch_cloud_metrics(subscription_id, resource_group, vm_name):
    try:
        # Define the resource ID
        resource_id = (
            f"/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.Compute/virtualMachines/{vm_name}"
        )

        print(f"Fetching metrics for resource ID: {resource_id}")

        # Query metrics
        response = client.query(
            resource_id,
            metric_names=["Percentage CPU"],
            timespan=datetime.timedelta(minutes=5),
            interval="PT1M",
            aggregations=[MetricAggregationType.AVERAGE]
        )

        datapoints = []
        if response.metrics:
            for result in response.metrics[0].timeseries[0].data:
                if result.average is not None:
                    datapoints.append({
                        "timestamp": result.timestamp,
                        "average": result.average
                    })
        else:
            print("No metrics available for the specified resource.")

        return datapoints
    except Exception as e:
        print(f"Error fetching metrics: {e}")
        return []

class Host:
    def __init__(self, id, cpu_capacity, memory_capacity):
        self.id = id
        self.cpu_capacity = cpu_capacity
        self.memory_capacity = memory_capacity
        self.available_cpu = cpu_capacity
        self.available_memory = memory_capacity
        self.vms = []

    def allocate_resources(self, vm):
        if self.available_cpu >= vm.cpu and self.available_memory >= vm.memory:
            self.available_cpu -= vm.cpu
            self.available_memory -= vm.memory
            self.vms.append(vm)
            print(f"VM {vm.id} allocated to Host {self.id}")
            return True
        return False

    def release_resources(self, vm):
        self.available_cpu += vm.cpu
        self.available_memory += vm.memory
        self.vms.remove(vm)
        print(f"VM {vm.id} released from Host {self.id}")

class VM:
    def __init__(self, id, cpu, memory, start_time, end_time):
        self.id = id
        self.cpu = cpu
        self.memory = memory
        self.start_time = start_time
        self.end_time = end_time
        self.host = None

class QLearningAgent:
    def __init__(self, alpha, gamma, epsilon):
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.q_table = {}

    def get_state(self, host, vm):
        return (host.id, vm.id)

    def choose_action(self, state):
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(['migrate', 'stay'])
        else:
            return max(self.q_table.get(state, {'migrate': 0, 'stay': 0}), key=self.q_table.get(state, {'migrate': 0, 'stay': 0}).get)

    def update_q_value(self, state, action, reward, next_state):
        old_value = self.q_table.get(state, {}).get(action, 0)
        future_reward = max(self.q_table.get(next_state, {}).values(), default=0)
        new_value = old_value + self.alpha * (reward + self.gamma * future_reward - old_value)
        if state not in self.q_table:
            self.q_table[state] = {}
        self.q_table[state][action] = new_value

def calculate_reward(host):
    utilization = (host.cpu_capacity - host.available_cpu) / host.cpu_capacity
    return utilization

def handle_vm_lifetime(env, vm, hosts, agent):
    for host in hosts:
        if host.allocate_resources(vm):
            vm.host = host
            break

    if vm.host is None:
        print(f"VM {vm.id} could not be allocated to any host")
        return

    lifetime = vm.end_time - vm.start_time
    yield env.timeout(lifetime)

    state = agent.get_state(vm.host, vm)
    action = agent.choose_action(state)

    if action == 'migrate':
        for host in hosts:
            if host != vm.host and host.allocate_resources(vm):
                vm.host.release_resources(vm)
                vm.host = host
                break

    reward = calculate_reward(vm.host)
    next_state = agent.get_state(vm.host, vm)
    agent.update_q_value(state, action, reward, next_state)

    vm.host.release_resources(vm)

def generate_vm_workload(env, subscription_id, resource_group, vm_name, hosts, agent):
    while True:
        print("Fetching metrics...")
        metrics = fetch_cloud_metrics(subscription_id, resource_group, vm_name)
        if not metrics:
            print("No metrics fetched.")
        else:
            for metric in metrics:
                start_time = metric['timestamp'].timestamp()
                end_time = start_time + 15  # Reduced interval for more real-time simulation
                cpu_demand = metric['average']
                memory_demand = 50  # Fixed memory demand for simplicity

                vm_id = int(start_time)
                vm = VM(vm_id, cpu_demand, memory_demand, start_time, end_time)
                env.process(handle_vm_lifetime(env, vm, hosts, agent))
                yield env.timeout(10)  # Wait for the next interval to fetch new metrics

def print_host_utilization(hosts):
    for host in hosts:
        utilization = (host.cpu_capacity - host.available_cpu) / host.cpu_capacity
        print(f'Host {host.id} utilization: {utilization:.2f}')

# Simulation parameters
simulation_duration = 1000  # Short duration for quick test
alpha = 0.1
gamma = 0.9
epsilon = 0.1

# Initialize environment and components
env = simpy.Environment()
hosts = [Host(i, 100, 100) for i in range(5)]  # Reduced number of hosts for simplicity
agent = QLearningAgent(alpha, gamma, epsilon)

# Example subscription ID, resource group, and VM name
subscription_id = 'd89493f7-fd93-418b-a882-dd9d2cb7c099'
resource_group = 'cat2_group'
vm_name = 'Hhi2003'

print("Starting simulation...")
env.process(generate_vm_workload(env, subscription_id, resource_group, vm_name, hosts, agent))
env.run(until=simulation_duration)
print("Simulation finished.")

print_host_utilization(hosts)


ModuleNotFoundError: No module named 'simpy'

In [ ]:
!pip install azure-monitor-query
!pip install azure-identity


In [ ]:
from azure.monitor.query import MetricsQueryClient


In [ ]:
!pip install azure-identity azure-monitor-query

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.1/166.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.3/162.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.3/194.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.2 MB/s eta 0:00:00


In [ ]:
!pip install chatterbot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.0/265.0 kB 17.3 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import chatterbot.comparisons
import chatterbot.response_selection
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
from chatterbot.conversation import Statement
# from spellchecker import SpellChecker
import logging

# def get_feedback():

#     text = input()

#     if 'yes' in text.lower():
#         return True
#     elif 'no' in text.lower():
#         return False
#     else:
#         print('Please type either "Yes" or "No"')
#         return get_feedback()

class Chat_Bot():

    def __init__(self):
        self.chatbot1 = ChatBot('Bakery Help',storage_adapter="chatterbot.storage.SQLStorageAdapter" ,logic_adapters=[
        {
                    'import_path': 'chatterbot.logic.BestMatch',
                    'default_response': 'I am sorry, but I do not understand.',
                    "statement_comparison_function": chatterbot.comparisons.LevenshteinDistance,
                    # "response_selection_method": chatterbot.response_selection.get_first_response
                    # 'maximum_similarity_threshold': 0.90
                }
            ]
            )
        logger=logging.getLogger()
        logger.setLevel(logging.CRITICAL)

        with open('greetings.txt','r') as data:
            welcome = data.read().splitlines()

        with open('asking_a_menu.txt','r') as ask:
            ask_menu = ask.read().splitlines()

        with open('seasonal_menu.txt','r') as ask1:
            seasonal_menu = ask1.read().splitlines()

        with open('gift_box.txt','r') as ask2:
            gift_box = ask2.read().splitlines()

        with open('macarons.txt','r') as d1:
            macaron = d1.read().splitlines()

        with open('chouquette.txt','r') as d2:
            chouquette = d2.read().splitlines()

        with open('creme_brulee.txt','r') as d3:
            creme_brulee = d3.read().splitlines()

        with open('croissant.txt','r') as d4:
            croissant = d4.read().splitlines()

        with open('eclair.txt','r') as d5:
            eclair = d5.read().splitlines()

        with open('mille_feuille.txt','r') as d6:
            mille_feuille = d6.read().splitlines()

        with open('pain_au_chocolat.txt','r') as d7:
            pain_au_chocolat = d7.read().splitlines()

        with open('pralines.txt','r') as d8:
            pralines = d8.read().splitlines()

        with open('profiterole.txt','r') as d9:
            profiterole = d9.read().splitlines()

        with open('extra_questions.txt','r') as d10:
            extra_questions = d10.read().splitlines()


        # trainer = ListTrainer(self.chatbot1)
        # trainer.train(welcome)
        # trainer.train(ask_menu)
        # trainer.train(seasonal_menu)
        # trainer.train(gift_box)
        # trainer.train(macaron)
        # trainer.train(chouquette)
        # trainer.train(creme_brulee)
        # trainer.train(croissant)
        # trainer.train(eclair)
        # trainer.train(mille_feuille)
        # trainer.train(pain_au_chocolat)
        # trainer.train(pralines)
        # trainer.train(profiterole)
        # trainer.train(extra_questions)


        # spell = SpellChecker()

        # print("Lets chat! Enter bye to exit")
    def get_chatbot_response(self,message):
        while True:
            try:
                input_statement = Statement(text=message)
                if "bye" in str(input_statement):
                    return "exit"
                elif "thank you" in str(input_statement):
                    response= "Is there anything else I can help you with ? Enter bye to exit"
                else:
                    response = self.chatbot1.generate_response(input_statement)

                    # print("\U0001F9C1 ",response.text)
                # print('\n Is "{}" a coherent response to "{}"? \n'.format(
                #     response.text,
                #     input_statement.text
                # ))
                # if get_feedback() is False:
                #     print('please input the correct one')
                #     correct_response = Statement(text=input())
                #     chatbot1.learn_response(correct_response, input_statement)
                #     print('Responses added to bot!')
                return response
            except (KeyboardInterrupt, EOFError, SystemExit):
                break




ModuleNotFoundError: No module named 'chatterbot'